In [52]:
from env import MyEnv
import ray
from ray.rllib.agents import ppo
from ray.tune.registry import register_env
import os
import numpy as np
from submission_utils import general_weights_fixer
from sklearn.preprocessing import MinMaxScaler

In [2]:
!nvidia-smi

Thu Feb 11 17:17:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:02:00.0 Off |                    0 |
| N/A   53C    P0    73W / 250W |   1389MiB / 16280MiB |     51%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:81:00.0 Off |                    0 |
| N/A   67C    P0    96W / 250W |  14649MiB / 16280MiB |     57%      Default |
+-------------------------------+----------------------+----------------------+
        

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


def env_creator(env_config):
    return MyEnv("observations_test_df.csv", "synthetic_assetsdf.csv", n_assets=96)

In [4]:
CONFIG = {
    'num_workers': 1,
    'num_envs_per_worker': 1,
    'create_env_on_driver': False,
    'rollout_fragment_length': 512,
    'batch_mode': 'truncate_episodes',
    'num_gpus': 1,
    'train_batch_size': 7680,
    'model': {"fcnet_hiddens": [2048, 1024, 512, 512],
    'fcnet_activation': "relu",
    'conv_filters': None,
    'conv_activation': 'relu',
    'free_log_std': False,
    'no_final_linear': False,
    'vf_share_layers': True,
    'use_lstm': False, # TODO: TURN TO TRUE 
    'max_seq_len': 256,
    'lstm_cell_size': 256,
    'lstm_use_prev_action_reward': False,
    '_time_major': False,
    'framestack': True,
    'dim': 84,
    'grayscale': False,
    'zero_mean': True,
    'custom_model': None,
    'custom_model_config': {},
    'custom_action_dist': None,
    'custom_preprocessor': None},
    'optimizer': {},
    'gamma': 0.99,
    'horizon': None,
    'soft_horizon': False,
    'no_done_at_end': False,
    'env_config': {},
    'env': "MyEnvTest",
    'normalize_actions': False,
    'clip_rewards': True,
    'clip_actions': True,
    'preprocessor_pref': 'deepmind',
    'lr': 5e-05,
    'monitor': False,
    'log_level': 'WARN',
    'ignore_worker_failures': True,
    'log_sys_usage': True,
    'fake_sampler': False,
    'framework': 'tf',
    'eager_tracing': False,
    'explore': True,
    'exploration_config': {'type': 'StochasticSampling'},
    'evaluation_interval': None,
    'evaluation_num_episodes': 10,
    'in_evaluation': False,
    'evaluation_config': {},
    'evaluation_num_workers': 0,
    'custom_eval_function': None,
    'sample_async': False,
    '_use_trajectory_view_api': True,
    'observation_filter': 'NoFilter',
    'synchronize_filters': True,
    'tf_session_args': {'intra_op_parallelism_threads': 2,
    'inter_op_parallelism_threads': 2,
    'gpu_options': {'allow_growth': True},
    'log_device_placement': False,
    'device_count': {'CPU': 1},
    'allow_soft_placement': True},
    'local_tf_session_args': {'intra_op_parallelism_threads': 8,
    'inter_op_parallelism_threads': 8},
    'compress_observations': False,
    'collect_metrics_timeout': 180,
    'metrics_smoothing_episodes': 100,
    'remote_worker_envs': False,
    'remote_env_batch_wait_ms': 0,
    'min_iter_time_s': 0,
    'timesteps_per_iteration': 0,
    'seed': None,
    'extra_python_environs_for_driver': {},
    'extra_python_environs_for_worker': {},
    'num_cpus_per_worker': 1,
    'num_gpus_per_worker': 0,
    'custom_resources_per_worker': {},
    'num_cpus_for_driver': 1,
    'memory': 0,
    'object_store_memory': 0,
    'memory_per_worker': 0,
    'object_store_memory_per_worker': 0,
    'input': 'sampler',
    'input_evaluation': ['is', 'wis'],
    'postprocess_inputs': False,
    'shuffle_buffer_size': 0,
    'output': None,
    'output_compress_columns': ['obs', 'new_obs'],
    'output_max_file_size': 67108864,
    'multiagent': {'policies': {},
    'policy_mapping_fn': None,
    'policies_to_train': None,
    'observation_fn': None,
    'replay_mode': 'independent'},
    'logger_config': None,
    'replay_sequence_length': 1,
    'use_critic': True,
    'use_gae': True,
    'lambda': 1.0,
    'kl_coeff': 0.2,
    'sgd_minibatch_size': 256,
    'shuffle_sequences': True,
    'num_sgd_iter': 30,
    'lr_schedule': None,
    'vf_share_layers': False,
    'vf_loss_coeff': 1.0,
    'entropy_coeff': 0.0,
    'entropy_coeff_schedule': None,
    'clip_param': 0.3,
    'vf_clip_param': 30000,
    "grad_clip": 1.0,
    'kl_target': 0.01,
    'simple_optimizer': False,
    '_fake_gpus': False
}

In [5]:
register_env("MyEnvTest", env_creator)


In [6]:
ray.init(num_gpus=1)


2021-02-11 17:17:35,765	INFO services.py:1182 -- View the Ray dashboard at http://127.0.0.1:8266


{'node_ip_address': '150.244.198.153',
 'raylet_ip_address': '150.244.198.153',
 'redis_address': '150.244.198.153:21505',
 'object_store_address': '/tmp/ray/session_2021-02-11_17-17-34_949471_23906/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-02-11_17-17-34_949471_23906/sockets/raylet',
 'webui_url': '127.0.0.1:8266',
 'session_dir': '/tmp/ray/session_2021-02-11_17-17-34_949471_23906',
 'metrics_export_port': 61058,
 'node_id': '2fd49f465b01c809b04a4046fefce60abb9a9f472f7568f7c499af9f'}

In [7]:
agent = ppo.PPOTrainer(config=CONFIG, env="MyEnvTest")


2021-02-11 17:17:41,312	INFO trainer.py:616 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-02-11 17:17:41,315	WARNING deprecation.py:34 -- DeprecationWarning: `model.lstm_use_prev_action_reward` has been deprecated. Use `model.lstm_use_prev_action and model.lstm_use_prev_reward` instead. This will raise an error in the future!
2021-02-11 17:17:41,318	INFO trainer.py:643 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=24275) WARNING:tensorflow:From /home/alejandro.vaca/miniconda/envs/series_temporales/lib/python3.7/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=24275) Instructions for updating:
(pid=24275) non-resource variables are not supported in the long term
(pid=24275) /home/alejandro.vaca/miniconda/envs/series_temporales/lib/python

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2021-02-11 17:18:15,608	WARNING deprecation.py:34 -- DeprecationWarning: `config[vf_share_layers]` has been deprecated. Use `config[model][vf_share_layers]` instead. This will raise an error in the future!
2021-02-11 17:18:17,923	WARNING deprecation.py:34 -- DeprecationWarning: `config[vf_share_layers]` has been deprecated. Use `config[model][vf_share_layers]` instead. This will raise an error in the future!
2021-02-11 17:18:23,212	INFO trainable.py:103 -- Trainable.setup took 41.906 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2021-02-11 17:18:23,216	WARNING util.py:47 -- Install gputil for GPU system monitoring.


In [ ]:
# PPO_MyEnv_0c3c8_00000

In [118]:
!ls ~/ray_results/PPO_SpainAI_1002/PPO_MyEnv_0c3c8_00000_0_2021-02-11_14-07-09/checkpoint_1015/checkpoint-1015


/home/alejandro.vaca/ray_results/PPO_SpainAI_1002/PPO_MyEnv_0c3c8_00000_0_2021-02-11_14-07-09/checkpoint_1015/checkpoint-1015


In [8]:
#!cat ~/ray_results/PPO_SpainAI_1002/PPO_MyEnv_77d40_00000_0_2021-02-10_17-38-56/params.json

In [38]:
128*15

1920

In [119]:
agent.restore("/home/alejandro.vaca/ray_results/PPO_SpainAI_1002/PPO_MyEnv_0c3c8_00000_0_2021-02-11_14-07-09/checkpoint_1015/checkpoint-1015")

2021-02-11 18:08:46,198	INFO trainable.py:372 -- Restored on 150.244.198.153 from checkpoint: /home/alejandro.vaca/ray_results/PPO_SpainAI_1002/PPO_MyEnv_0c3c8_00000_0_2021-02-11_14-07-09/checkpoint_1015/checkpoint-1015
2021-02-11 18:08:46,199	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 1015, '_timesteps_total': None, '_time_total': 13562.80629491806, '_episodes_total': 585}


In [120]:
env = MyEnv("observations_test_df.csv", "synthetic_assetsdf.csv", n_assets=96)

In [13]:
obs = env.reset()

In [14]:
obs.shape

(110,)

In [106]:
#dir(agent)

In [16]:
policy = agent.get_policy()

In [105]:
#policy.compute_actions([obs])

In [49]:
#np.clip([-1, -0.0000005, 0.0, 1.000000001], 0, 1)

array([0., 0., 0., 1.])

In [53]:
scaler = MinMaxScaler()

In [121]:
def process_action(action):
    #action = np.clip(action, 0.0, 1.0)
    #action = scaler.fit_transform(action.reshape(-1,1)).reshape((96, ))
    action = np.clip(action, 0, np.inf)
    #action = np.array([act if act > 0 else 0 for act in action])
    action = action / sum(action)
    return action

In [122]:
actions = []
done = False
obs = env.reset()
while not done:
    action = agent.compute_action(obs)
    obs, reward, done, info = env.step(action)
    actions.append(process_action(action))

/home/alejandro.vaca/reto_series_temporales/env.py:63: RuntimeWarning: divide by zero encountered in log
  returns = np.diff(np.log(self.wallet_values))
/home/alejandro.vaca/miniconda/envs/series_temporales/lib/python3.7/site-packages/numpy/lib/function_base.py:1280: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
/home/alejandro.vaca/miniconda/envs/series_temporales/lib/python3.7/site-packages/empyrical/stats.py:711: RuntimeWarning: Mean of empty slice
  nanmean(returns_risk_adj, axis=0),
/home/alejandro.vaca/miniconda/envs/series_temporales/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1665: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


In [123]:
len(actions)

2228

In [116]:
#actions[:2]

In [81]:
any([sum(act) < 1 for act in actions])

True

In [82]:
any([sum(act) > 1 for act in actions])

True

In [83]:
np.argmax(actions[0])

4

In [84]:
actions2 = [actions[0]] + actions

In [85]:
len(actions2)

2229

In [125]:
actions3 = list(map(general_weights_fixer, actions2))

In [126]:
all([sum(act) == 1 for act in actions3])

True

In [127]:
any([sum(act) < 1 for act in actions3])

False

In [128]:
any([sum(act) > 1 for act in actions3])

False

In [130]:
any([a < 0  for act in actions3 for a in act])

False

In [124]:
def general_weights_fixer(lista):
    lista = [round(act, 3) for act in lista]
    suma = sum(lista)
    if suma > 1:
        sobra = suma - 1
        idx = np.argmax(lista)
        prev = lista[idx]
        lista[idx] -= sobra
    elif suma < 1:
        falta = 1 - suma
        idx = np.argmax(lista)
        prev = lista[idx]
        lista[idx] += falta
    return lista

In [103]:
import pandas as pd
ss = pd.read_csv("synthetic_assetsdf.csv", parse_dates=["date"], index_col="date")

In [104]:
ss.columns

Index(['ZVQ_close', 'NCT_close', 'YAX_close', 'OOS_close', 'GFJ_close',
       'FIR_close', 'USX_close', 'FSK_close', 'TMF_close', 'TDD_close',
       'HEO_close', 'ULT_close', 'CSB_close', 'CIS_close', 'UYZ_close',
       'TRO_close', 'YEC_close', 'ERO_close', 'ZAB_close', 'HZY_close',
       'MUF_close', 'ZTY_close', 'PUL_close', 'JHI_close', 'LWE_close',
       'SEH_close', 'AWW_close', 'GRI_close', 'REU_close', 'ACY_close',
       'MMY_close', 'VRT_close', 'ZXW_close', 'LUG_close', 'VVC_close',
       'CBY_close', 'WFJ_close', 'PPT_close', 'PEW_close', 'MCA_close',
       'IDT_close', 'FFV_close', 'LWK_close', 'ZCD_close', 'ULI_close',
       'AUX_close', 'DZF_close', 'NWO_close', 'BAX_close', 'HCC_close',
       'JNE_close', 'JTL_close', 'UPP_close', 'WXN_close', 'XRX_close',
       'UEI_close', 'LHB_close', 'NYD_close', 'BGN_close', 'SHC_close',
       'THA_close', 'TKT_close', 'GRU_close', 'BFS_close', 'SKN_close',
       'NVL_close', 'YFC_close', 'GGR_close', 'TER_close', 'NYP_

In [107]:
subm = pd.read_csv("submission/submission.csv")

In [108]:
subm.head()

,eod_ts,allo_REU,allo_VRT,allo_EEY,allo_JTL,allo_SEH,allo_BSX,allo_OJG,allo_UEI,allo_HQU,allo_ZXW,allo_LEN,allo_YEC,allo_UYZ,allo_LWK,allo_ACY,allo_HEO,allo_FIR,allo_BGN
0,2020-08-18 00:00:00,0.114379,0.029099,0.013186,0.101076,0.101519,0.057257,0.027439,0.018777,0.053406,0.080812,0.020080,0.087687,0.070070,0.099456,0.029514,0.037835,0.016323,0.042084
1,2020-08-18 01:00:00,0.014112,0.088849,0.022711,0.026985,0.003099,0.040348,0.045789,0.121986,0.114450,0.107733,0.024035,0.043220,0.093317,0.008298,0.086308,0.065200,0.041210,0.052349
2,2020-08-18 02:00:00,0.069645,0.046509,0.026061,0.086234,0.045954,0.058223,0.046801,0.045134,0.121548,0.014297,0.067253,0.083320,0.043973,0.090604,0.016118,0.120136,0.016471,0.001718
3,2020-08-18 03:00:00,0.107072,0.035263,0.106222,0.007086,0.055552,0.042929,0.093544,0.069414,0.014073,0.037031,0.023287,0.088242,0.049342,0.074206,0.069467,0.104633,0.020961,0.001674
4,2020-08-18 04:00:00,0.060190,0.069420,0.027955,0.077568,0.062527,0.049295,0.003327,0.006662,0.019053,0.075851,0.075360,0.079179,0.023196,0.082323,0.090402,0.097320,0.015146,0.085227


In [131]:
def get_submission(actions, index, reference_df):
    cols =  [f"allo_{col.replace('_close', '')}" for col in reference_df.columns]
    df = pd.DataFrame(actions, columns=cols)
    df.insert(0, "eod_ts", index)
    return df

In [132]:
submission = get_submission(actions3, subm["eod_ts"], ss)

In [133]:
submission.head()

,eod_ts,allo_ZVQ,allo_NCT,allo_YAX,allo_OOS,allo_GFJ,allo_FIR,allo_USX,allo_FSK,allo_TMF,...,allo_AZG,allo_OJG,allo_WWT,allo_BOT,allo_TXR,allo_RAT,allo_DIG,allo_SRI,allo_PHI,allo_RWJ
0,2020-08-18 00:00:00,0.000,0.000,0.000,0.0,0.029,0.000,0.000,0.000,0.026,...,0.00,0.000,0.026,0.000,0.000,0.004,0.026,0.000,0.026,0.018
1,2020-08-18 01:00:00,0.000,0.000,0.000,0.0,0.029,0.000,0.000,0.000,0.026,...,0.00,0.000,0.026,0.000,0.000,0.004,0.026,0.000,0.026,0.018
2,2020-08-18 02:00:00,0.000,0.018,0.000,0.0,0.008,0.000,0.031,0.000,0.014,...,0.03,0.001,0.000,0.030,0.009,0.004,0.030,0.021,0.012,0.000
3,2020-08-18 03:00:00,0.020,0.002,0.033,0.0,0.000,0.014,0.000,0.023,0.006,...,0.00,0.000,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000
4,2020-08-18 04:00:00,0.036,0.009,0.000,0.0,0.000,0.016,0.000,0.000,0.000,...,0.00,0.000,0.030,0.006,0.000,0.009,0.000,0.000,0.030,0.000


In [134]:
submission.to_csv("submission_1102_2.csv", header=True, index=False)